# Demo cas test avec maquette

Ceci est un notebook pour des expériences numériques en différents cas faites avec la maquette.

Cas infiltration, conduction : dH > 0, K petit

Cas infiltration, advection : dH > 0, K grand

Cas exfiltration, conduction : dH < 0, K petit

Cas exfiltration, advection : dH < 0, K grand

In [1]:
from pyheatmy import *
import matplotlib.pyplot as plt
import numpy as np

## Variables globales de l'expérience

In [2]:
K_offset = 273.15

t_debut = (2010, 1, 1)
t_fin = (2010, 5, 30, 23, 59, 59)
dt = 15*60 # pas de temps en (s)

T_riv_amp = 1
T_riv_offset = 20 + K_offset
P_T_riv = 24*4*dt

T_aq_amp = 0
T_aq_offset = 12 + K_offset
P_T_aq = -9999 # à mettre dans le init

dH_amp = 0
dH_offset = 1
P_dh = -9999 #14*24*4*dt

## Fonction de définition de l'expérience

In [12]:
def gen_experiment_monolayer(depth_sensors, name, moinslog10K, n, lambda_s, rho_cs, nb_cells, perturb="False"):
    """Cette fonction va dépendre de beaucoup de variables globales"""
    # définition de l'expérience 
    """Times series"""
    time_series_dict = {
        "offset" : 0,
        "depth_sensors":depth_sensors,
        "param_time_dates": [t_debut, t_fin, dt], 
        "param_dH_signal": [dH_amp, P_dh, dH_offset],
        "param_T_riv_signal": [T_riv_amp, P_T_riv, T_riv_offset],
        "param_T_aq_signal": [T_aq_amp, P_T_aq, T_aq_offset],
        "sigma_meas_P": 0.05,
        "sigma_meas_T": 0.1, #float
    }
    # on génère l'objet Times_series
    time_series = Time_series.from_dict(time_series_dict)
    # on génère les données
    if perturb == True :
        time_series._generate_perturb_T_riv_dH_series()
        time_series._generate_perturb_T_Shaft_Temp_series()
    else :
        time_series._generate_T_riv_dH_series()
        time_series._generate_Shaft_Temp_series()

    """Colonne"""
    col_dict = {
        "river_bed": 1., 
        "depth_sensors": depth_sensors,
        "offset": .0,
        "dH_measures": time_series._T_riv_dH_measures,
        "T_measures": time_series._T_Shaft_measures,
        "sigma_meas_P": time_series._sigma_P, #float
        "sigma_meas_T": time_series._sigma_T, #float
    }
    # on génère l'objet Column à partir de l'objet Time_series
    colonne_exp = Column.from_dict(col_dict)

    """Layer"""
    monolayer_dict = {
        "name": name,
        "zLow": depth_sensors[-1],
        "moinslog10K": moinslog10K,
        "n": n,
        "lambda_s": lambda_s,
        "rhos_cs": rho_cs
    }
    # on génère l'objet Layer
    monolayer = Layer.from_dict(monolayer_dict)

    """Valeurs analytiques"""
    analy_sol_dict = {
        "column_exp" : colonne_exp,
        "time_series" : time_series,
        "monolayer" : monolayer,
        "nb_cells" : nb_cells,
    }

    # on génère l'objet des solution analytique
    analy_sol_exp = Analy_Sol.from_dict(analy_sol_dict)
    return time_series, colonne_exp, monolayer, analy_sol_exp

## 1. Maquette multilayer (Billes en verre, Sable)


In [13]:
# paramètres principaux de l'expérience
dict_exp ={
    "depth_sensors": [.25, .5, .75, 1],
    "name" : "sable",
    "moinslog10K": 4,
    "n": 0.1,
    "lambda_s": 2,
    "rho_cs": 4e6,
    "nb_cells": 100,
}
# maj variables globales

experiment = [*gen_experiment_monolayer(**dict_exp)] # [time_series, colonne_exp, monolayer, analy_sol_exp]

#### d. Schéma maquette

à ajouter 

### B. Calculs et prédictions d'obeservation

In [ ]:
col.compute_solve_transi(layersList_1, 100)

In [ ]:
# Diagramme T-z
fig, ax = plt.subplots(figsize=(10, 7), facecolor = 'w')

imm = ax.imshow(
    col.temps_solve - 273.15,
    aspect = "auto",
    extent = [0, 1, col.depths_solve[-1], col.depths_solve[0]], #TODO : trouver comment monter le datetime
)
cbar = fig.colorbar(imm, shrink=1,location='left')
cbar.set_label('Temperature (Celsius)')

fig.suptitle("Variantion of temperature")
ax.set_xlabel("time (?)")
ax.set_ylabel("depth (m)")
np.min(col.temps_solve)

In [ ]:
# Diagramme T-t
ind1 = 25
ind2 = 50
ind3 = 75
print(f"Index {ind1} in z_array -> {col.depths_solve[ind1]}")
print(f"Index {ind2} in z_array -> {col.depths_solve[ind2]}")
print(f"Index {ind3} in z_array -> {col.depths_solve[ind3]}")

fig, ax = plt.subplots(facecolor = 'w')
ax.plot(col.times_solve, col.get_temps_solve(col.depths_solve[0])-273.15, label = f"depth={col.depths_solve[0]:.3f}m")
ax.plot(col.times_solve, col.get_temps_solve(col.depths_solve[ind1])-273.15, label = f"depth={col.depths_solve[ind1]:.3f}m")
ax.plot(col.times_solve, col.get_temps_solve(col.depths_solve[ind2])-273.15, label = f"depth={col.depths_solve[ind2]:.3f}m")
ax.plot(col.times_solve, col.get_temps_solve(col.depths_solve[ind3])-273.15, label = f"depth={col.depths_solve[ind3]:.3f}m")
fig.suptitle("Variation of temperature at different depths")
ax.set_xlabel("time (s)")
ax.set_ylabel("temperature (K)")
#ax.set_xlim(0,0.1)
ax.legend()

## 2.Monolayer stationnaire

### A. Cas infiltration conduction ：dH > 0, K petit

#### a. Définition d'expérience

In [ ]:
K_offset = 273.15

t_debut = (2010, 1, 1)
t_fin = (2010, 5, 30, 23, 59, 59)
dt = 15*60 # pas de temps en (s)

T_riv_amp = 1
T_riv_offset = 20 + K_offset
P_T_riv = 24*4*dt

T_aq_amp = 0
T_aq_offset = 12 + K_offset
P_T_aq = -9999 # à mettre dans le init

dH_amp = 0
dH_offset = 1
P_dh = -9999 #14*24*4*dt

In [ ]:
time_series_dict_user1 = {
    "offset":.0,
    "depth_sensors":[.25, .5, .75, 1],
	"param_time_dates": [t_debut, t_fin, dt], 
    "param_dH_signal": [dH_amp, P_dh, dH_offset], #En vrai y aura une 4e valeur ici mais ca prendra en charge pareil
	"param_T_riv_signal": [T_riv_amp, P_T_riv, T_riv_offset],
    "param_T_aq_signal": [T_aq_amp, P_T_aq, T_aq_offset],
    "sigma_meas_P": None,
    "sigma_meas_T": None, #float
}

emu_observ_test_user1 = Time_series.from_dict(time_series_dict_user1)
emu_observ_test_user1._dates

In [ ]:
from datetime import datetime

times = [datetime.fromtimestamp(1000*k) for k in range(500)]
temps = np.load("data/val_data.npy")

col_dict = {
	"river_bed": 1., 
    "depth_sensors": [.25, .5, .75, 1], #En vrai y aura une 4e valeur ici mais ca prendra en charge pareil
	"offset": .0,
    "dH_measures": list(zip(times,list(zip(-.01*np.cos(np.linspace(0,2*np.pi,500))+.03, temps[:,0])))),
	"T_measures": list(zip(times, temps[:,[32,66,-1]])),
    "sigma_meas_P": None, #float
    "sigma_meas_T": None, #float
}

col = Column.from_dict(col_dict)

In [ ]:
params = Param(
    moinslog10K = 12,
    n = .1,
    lambda_s = 2,
    rhos_cs = 4e6
)

#### b. Calculs

In [ ]:
col.compute_solve_transi(params, 100)

## 3. Multilayer : 1 capteur par layer

## 4. Multilayer : 1 layer sans capteur